In [1]:
import os
import numpy as np
import torchvision.datasets as datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam
from torchvision import models
import torchvision.transforms as transforms
from misc_functions import preprocess_image, recreate_image, save_image
import matplotlib.pyplot as plt
import matplotlib

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Number of GPUs available: ',torch.cuda.device_count())
device

Number of GPUs available:  1


device(type='cuda', index=0)

In [3]:
class Data():
    
    def __init__(self):
        
        self.train_set = datasets.CIFAR10(
        root = './data',
        train = True,
        download = True,
        transform = transforms.Compose([transforms.Resize(224), transforms.ToTensor()])
        )

        self.test_set = datasets.CIFAR10(
        root= './data',
        train= False,
        download = True,
        transform = transforms.Compose([transforms.Resize(224), transforms.ToTensor()])
        )
        
    def get_data(self):
        return [self.train_set, self.test_set]

In [4]:
data = Data()
train, test = data.get_data()
train_loader = torch.utils.data.DataLoader(train, batch_size= 256, shuffle= True)
test_loader = torch.utils.data.DataLoader(test, batch_size= 10000, shuffle= True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
images, labels = next(iter(train_loader))
images.shape

torch.Size([256, 3, 224, 224])

In [6]:
model = models.vgg16_bn(pretrained= True)
model.to(device)
model.classifier[6].out_features = 10
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3)

In [7]:
count = 0
for params in model.parameters():
    count += 1
    if count < 57:
        params.requires_grad = False


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
#optimizer = optim.Adam(net.parameters(), lr=0.001)
def train(epoch, losses, net):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        losses.append(loss.item())
        _, predicted = outputs.max(1)  #this returns values and labels
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print(batch_idx, len(train_loader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
def test(net, test_losses ):
    
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            
            test_losses.append(loss.item())
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print('Loss: %.3f | Acc: %.3f%%'
                  % (test_loss, 100.*correct/len(test_loader.dataset)))

In [ ]:
epochs = 3
losses = []
test_losses = []
for epoch in range(epochs):
    train(epoch, losses, model)


Epoch: 0
0 196 Loss: 9.664 | Acc: 0.781% (2/256)
1 196 Loss: 9.645 | Acc: 0.391% (2/512)
2 196 Loss: 9.657 | Acc: 0.260% (2/768)
3 196 Loss: 9.595 | Acc: 0.195% (2/1024)
4 196 Loss: 9.535 | Acc: 0.156% (2/1280)
5 196 Loss: 9.490 | Acc: 0.260% (4/1536)
6 196 Loss: 9.426 | Acc: 0.223% (4/1792)
7 196 Loss: 9.379 | Acc: 0.195% (4/2048)
8 196 Loss: 9.351 | Acc: 0.174% (4/2304)
9 196 Loss: 9.306 | Acc: 0.234% (6/2560)
10 196 Loss: 9.206 | Acc: 0.249% (7/2816)
11 196 Loss: 9.125 | Acc: 0.228% (7/3072)
12 196 Loss: 9.038 | Acc: 0.210% (7/3328)
13 196 Loss: 8.967 | Acc: 0.251% (9/3584)
14 196 Loss: 8.886 | Acc: 0.286% (11/3840)
15 196 Loss: 8.797 | Acc: 0.293% (12/4096)
16 196 Loss: 8.720 | Acc: 0.276% (12/4352)
17 196 Loss: 8.633 | Acc: 0.391% (18/4608)
18 196 Loss: 8.538 | Acc: 0.493% (24/4864)
19 196 Loss: 8.451 | Acc: 0.723% (37/5120)
20 196 Loss: 8.362 | Acc: 0.800% (43/5376)
21 196 Loss: 8.264 | Acc: 0.977% (55/5632)
22 196 Loss: 8.171 | Acc: 1.138% (67/5888)
23 196 Loss: 8.074 | Acc: 1.

In [ ]:
log_loss = [np.log(x) for x in losses]
plt.plot(log_loss)

plt.ylabel('Loss')
plt.xlabel('Epochs')

In [ ]:
model.classifier = nn.Sequential(nn.Linear(in_features= 25088, out_features= 4096 ),
                                nn.ReLU(),
                                nn.Dropout2d(),
                                nn.Linear(in_features= 4096, out_features= 4096),
                                nn.ReLU(),
                                nn.Dropout2d(),
                                nn.Linear(in_features= 4096, out_features= 10))

In [ ]:
class CNNLayerVisualization():
    """
        Produces an image that minimizes the loss of a convolution
        operation for a specific layer and filter
    """
    def __init__(self, model, selected_layer, selected_filter):
        self.model = model
        self.model.eval()
        self.selected_layer = selected_layer
        self.selected_filter = selected_filter
        self.conv_output = 0
        # Create the folder to export images if not exists
        if not os.path.exists('generated'):
            os.makedirs('generated')

    def hook_layer(self):
        def hook_function(module, grad_in, grad_out):
            # Gets the conv output of the selected filter (from selected layer)
            print(grad_out[0, self.selected_filter].shape)
            self.conv_output = grad_out[0, self.selected_filter]
        # Hook the selected layer
        self.model[self.selected_layer].register_forward_hook(hook_function)

    def visualise_layer_with_hooks(self):
        # Hook the selected layer
        self.hook_layer()
        # Generate a random image
        #random_image = np.uint8(np.random.uniform(150, 180, (224, 224, 3)))
        random_image = images[3].permute(2,1,0)
        # Process image and return variable
        print("random image ", random_image.shape)
        processed_image = preprocess_image(random_image, False)
        print("processed images size", processed_image.shape)
        # Define optimizer for the image
        optimizer = Adam([processed_image], lr=0.1, weight_decay=1e-6)
        for i in range(1, 145):
            optimizer.zero_grad()
            # Assign create image to a variable to move forward in the model
            x = processed_image
            for index, layer in enumerate(self.model):
                # Forward pass layer by layer
                # x is not used after this point because it is only needed to trigger
                # the forward hook function
                x = layer(x)
                # Only need to forward until the selected layer is reached
                if index == self.selected_layer:
                    # (forward hook function triggered)
                    break
            # Loss function is the mean of the output of the selected layer/filter
            # We try to minimize the mean of the output of that specific filter
            loss = -torch.mean(self.conv_output)
            print('Iteration:', str(i), 'Loss:', "{0:.2f}".format(loss.data))
            # Backward
            loss.backward()
            # Update image
            optimizer.step()
            # Recreate image
            self.created_image = recreate_image(processed_image)
            # Save image
            if i % 5 == 0:
                im_path = 'generated/layer_vis_l' + str(self.selected_layer) + \
                    '_f' + str(self.selected_filter) + '_iter' + str(i) + '.jpg'
                save_image(self.created_image, im_path)

In [ ]:
cnn_layer = 17
filter_pos = 5
# Fully connected layer is not needed
pretrained_model = models.vgg16(pretrained=True).features
layer_vis = CNNLayerVisualization(pretrained_model, cnn_layer, filter_pos)

# Layer visualization with pytorch hooks
layer_vis.visualise_layer_with_hooks()

In [ ]:


print("dim: ", images[3][0].shape)
fig = plt.figure()
plt.imshow(images[3][0], interpolation= 'bicubic')
fig

In [ ]:
 models.vgg16(pretrained=True).features

In [ ]:

fig = plt.figure()
plt.imshow(layer_vis.created_image[:,:,0], interpolation= 'bicubic')
fig


In [ ]:
pretrained_model[17]
images[1].permute(2,1,0).shape